
Import libraries

In [1]:
from random import uniform
import bs4 as bs
import urllib.request
import numpy as np
import pandas as pd 
import folium
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import time

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
def scrape_url(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Store data to this temporary dataframe
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

In [4]:
raw_data = scrape_url("wikitable",3)
postal_codes_df=raw_data[~raw_data['Borough'].isin(['Not assigned'])]
postal_codes_df.loc[postal_codes_df['Neighbourhood'] == 'Not assigned','Neighbourhood'] = postal_codes_df[postal_codes_df['Neighbourhood'] == 'Not assigned']['Borough']
# THIS CALL THROWS A WARNING, PLEASE IGNORE IT
postal_codes_df = postal_codes_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
postal_codes_df.head()
#!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
filename = 'toronto_data.csv'
headers = ['Postal Code', 'Latitude', 'Longitude']
df = pd.read_csv(filename)
neigborhoods_w_coords = pd.merge(postal_codes_df, df, left_on='Postcode', right_on='Postal Code')
neigborhoods_w_coords.drop('Postal Code', axis=1, inplace=True)
neigborhoods_w_coords.head()

Data downloaded!


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [5]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for Latitude, Longitude, Borough, Neighborhood in zip(neigborhoods_w_coords['Latitude'], neigborhoods_w_coords['Longitude'], neigborhoods_w_coords['Borough'], neigborhoods_w_coords['Neighbourhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
CLIENT_ID = 'L4TEIJMILCEPTFI34OPPLJYNRPLUFD4BZZMJUKA1Z0DD2TFH' # your Foursquare ID
CLIENT_SECRET = '5QVK35FPKSKTNVOGJTKXCGC4RDAQUNKZHVZZCKZ52ARK2C5K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L4TEIJMILCEPTFI34OPPLJYNRPLUFD4BZZMJUKA1Z0DD2TFH
CLIENT_SECRET:5QVK35FPKSKTNVOGJTKXCGC4RDAQUNKZHVZZCKZ52ARK2C5K


In [10]:
search_query = 'grocery'
radius = 1000
LIMIT = 100
intent = 'browse'
categories = '4d4b7105d754a06374d81259,4d4b7105d754a06374d81259,52f2ab2ebcbc57f1066b8b1c,\
50be8ee891d4fa8dcc7199a7,4bf58dd8d48988d1fa941735,4bf58dd8d48988d10e951735,52f2ab2ebcbc57f1066b8b46,\
4e67e38e036454776db1fb3a'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&\
categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, categories, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=L4TEIJMILCEPTFI34OPPLJYNRPLUFD4BZZMJUKA1Z0DD2TFH&client_secret=5QVK35FPKSKTNVOGJTKXCGC4RDAQUNKZHVZZCKZ52ARK2C5K&ll=43.653963,-79.387207&v=20180605&categoryId=4d4b7105d754a06374d81259,4d4b7105d754a06374d81259,52f2ab2ebcbc57f1066b8b1c,50be8ee891d4fa8dcc7199a7,4bf58dd8d48988d1fa941735,4bf58dd8d48988d10e951735,52f2ab2ebcbc57f1066b8b46,4e67e38e036454776db1fb3a&radius=1000&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9fa221db04f53b15a6c597'},
 'response': {'venues': [{'id': '4ae60299f964a52003a421e3',
    'name': 'Starbucks',
    'location': {'address': '180 Queen St W,Suite 102.3A',
     'crossStreet': 'at Simcoe St.',
     'lat': 43.65071767190139,
     'lng': -79.3875952564609,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65071767190139,
       'lng': -79.3875952564609}],
     'distance': 362,
     'postalCode': 'M5V 3X3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['180 Queen St W,Suite 102.3A (at Simcoe St.)',
      'Toronto ON M5V 3X3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1553965601',

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
    except:
        categories_list = row['venue.categories']    
        if len(categories_list) == 0:
            return None
        else:
            return [categories_list[0]['name'], categories_list[0]['id']]

In [13]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.shape
# filter columns
filtered_columns = ['id','name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.tail()

,id,name,categories,lat,lng
45,56ec7523498e73779c59c037,Mike's Independent City Market Toronto,Market,43.648011,-79.393130
46,4ae0cf40f964a520bc8221e3,Jack Astor's Bar & Grill,American Restaurant,43.645315,-79.383837
47,59a0c566fc9e941199511d9f,Kellys Landing,Restaurant,43.645006,-79.382660
48,4f0b0b43e4b05f83101ee474,Starbucks,Coffee Shop,43.649028,-79.381593
49,4bf438f0e5eba5936d671f90,Starbucks,Coffee Shop,43.647193,-79.380957


In [14]:
search_query = 'grocery'
LIMIT = 100
radius=500
intent = 'browse'
categories = '4d4b7105d754a06374d81259,4d4b7105d754a06374d81259,52f2ab2ebcbc57f1066b8b1c,\
50be8ee891d4fa8dcc7199a7,4bf58dd8d48988d1fa941735,4bf58dd8d48988d10e951735,52f2ab2ebcbc57f1066b8b46,\
4e67e38e036454776db1fb3a'
# url = 
tempres = []

def getNearbyVenues(names):
    
    venues_list=[]
    for index, row in names.iterrows():
        print(row[0])
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&\
categoryId={}&radius={}&limit={}'.format(CLIENT_ID, 
                         CLIENT_SECRET, 
                         row[1], 
                         row[2], 
                         VERSION, 
                         categories, 
                         radius, 
                         LIMIT)
        txt = 'name: {}, coords: {},{}, category: {}'    
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
#         time.sleep(5)

        # return only relevant information for each nearby venue
        for v in results:
            venues_list.append([(
                row[0], 
                row[1], 
                row[2], 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'],  
                v['categories'][0]['name'])])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print('DONE!')
    return(nearby_venues)

In [7]:
# toronto_venues = getNearbyVenues(neigborhoods_w_coords.loc[:,['Neighbourhood','Latitude','Longitude']])
# toronto_venues.to_csv('toronto_venues.csv',index=False)
toronto_venues = pd.read_csv('toronto_venues.csv')

In [8]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Second Cup,43.802165,-79.196114,Coffee Shop
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Shamrock Burgers,43.783823,-79.168406,Burger Joint
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Tony Stacey Centre for Veterans Care,43.783857,-79.164738,Assisted Living


In [23]:
for index, data in toronto_venues.iterrows():
    toronto_venues.at[index,'rating'] = round(uniform(0,10), 2)
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
0,"Rouge, Malvern",43.806686,-79.194353,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,9.50
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,8.52
2,"Rouge, Malvern",43.806686,-79.194353,Second Cup,43.802165,-79.196114,Coffee Shop,3.06
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Shamrock Burgers,43.783823,-79.168406,Burger Joint,5.40
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Tony Stacey Centre for Veterans Care,43.783857,-79.164738,Assisted Living,4.64


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
Neighborhood,,,,,,,
"Adelaide, King, Richmond",50,50,50,50,50,50,50
Agincourt,8,8,8,8,8,8,8
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",20,20,20,20,20,20,20
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",37,37,37,37,37,37,37
"Alderwood, Long Branch",25,25,25,25,25,25,25
"Bathurst Manor, Downsview North, Wilson Heights",25,25,25,25,25,25,25
Bayview Village,12,12,12,12,12,12,12
"Bedford Park, Lawrence Manor East",48,48,48,48,48,48,48
Berczy Park,50,50,50,50,50,50,50


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Assisted Living,BBQ Joint,Bagel Shop,Bakery,Bar,...,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Trailer Park,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_onehot.shape

(2580, 152)

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
# toronto_grouped = toronto_venues.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,Assisted Living,BBQ Joint,Bagel Shop,Bakery,Bar,...,Tea Room,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Trailer Park,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0,0,4,0,0,0,0,0,2,...,0,1,0,0,0,0,0,0,0,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,0,2,0,3,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",1,0,0,0,0,0,1,5,0,...,0,0,0,0,0,0,0,0,0,0
4,"Alderwood, Long Branch",0,0,0,2,0,2,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [39]:
toronto_grouped.shape

(101, 152)

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  13.0
1  American Restaurant   4.0
2           Restaurant   3.0
3                 Café   3.0
4     Ramen Restaurant   3.0


----Agincourt----
                venue  freq
0      Sandwich Place   2.0
1         Coffee Shop   2.0
2  Chinese Restaurant   1.0
3   Korean Restaurant   1.0
4      Breakfast Spot   1.0


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0    Chinese Restaurant   4.0
1             BBQ Joint   3.0
2      Asian Restaurant   2.0
3                Bakery   2.0
4  Fast Food Restaurant   2.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  10.0
1                Bakery   5.0
2  Caribbean Restaurant   3.0
3     Indian Restaurant   3.0
4  Fast Food Restaurant   2.0


----Alderwood, Long Branch----
              venue  freq

In [26]:
toronto_venues.loc[:,['Neighborhood','Venue Category','rating']].groupby(['Neighborhood', 'Venue Category']).mean().reset_index()

,Neighborhood,Venue Category,rating
0,"Adelaide, King, Richmond",American Restaurant,4.132500
1,"Adelaide, King, Richmond",Bar,4.855000
2,"Adelaide, King, Richmond",Brazilian Restaurant,2.530000
3,"Adelaide, King, Richmond",Breakfast Spot,5.670000
4,"Adelaide, King, Richmond",Burger Joint,8.660000
5,"Adelaide, King, Richmond",Café,4.696667
6,"Adelaide, King, Richmond",Coffee Shop,5.536154
7,"Adelaide, King, Richmond",Deli / Bodega,1.880000
8,"Adelaide, King, Richmond",Fast Food Restaurant,6.365000
9,"Adelaide, King, Richmond",Food Court,3.500000


In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,American Restaurant,Café,Ramen Restaurant,Restaurant,Fast Food Restaurant,Gastropub,Hotel,Bar,Deli / Bodega
1,Agincourt,Coffee Shop,Sandwich Place,Restaurant,Korean Restaurant,Breakfast Spot,Chinese Restaurant,Wings Joint,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,BBQ Joint,Fast Food Restaurant,Asian Restaurant,Bakery,Pizza Place,Medical Center,Dumpling Restaurant,Caribbean Restaurant,Chinese Breakfast Place
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Bakery,Caribbean Restaurant,Indian Restaurant,Fast Food Restaurant,Mexican Restaurant,Residential Building (Apartment / Condo),Chinese Restaurant,Coffee Shop,Afghan Restaurant
4,"Alderwood, Long Branch",Coffee Shop,Breakfast Spot,Asian Restaurant,BBQ Joint,Pizza Place,Café,New American Restaurant,Market,Deli / Bodega,Chinese Restaurant


In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 1, 0, 4, 3, 0], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",43.806686,-79.194353,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,9.50,0,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Wings Joint,Diner,Egyptian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,8.52,0,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Wings Joint,Diner,Egyptian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Second Cup,43.802165,-79.196114,Coffee Shop,3.06,0,Coffee Shop,Fast Food Restaurant,Indian Restaurant,Wings Joint,Diner,Egyptian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Shamrock Burgers,43.783823,-79.168406,Burger Joint,5.40,0,African Restaurant,Assisted Living,Breakfast Spot,Burger Joint,Wings Joint,Doner Restaurant,English Restaurant,Empanada Restaurant,Egyptian Restaurant,Eastern European Restaurant
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Tony Stacey Centre for Veterans Care,43.783857,-79.164738,Assisted Living,4.64,0,African Restaurant,Assisted Living,Breakfast Spot,Burger Joint,Wings Joint,Doner Restaurant,English Restaurant,Empanada Restaurant,Egyptian Restaurant,Eastern European Restaurant


In [36]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape

(1767, 15)

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape

(3, 15)

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape

(130, 15)

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape

(679, 15)

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].shape

(1, 15)